In [1]:
import requests,time,os,csv,re
from bs4 import BeautifulSoup

url_base='http://sh.lianjia.com/ershoufang/d'

In [2]:
def get_html(url):
    try:
        r=requests.get(url,timeout=16)
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        return r.text
    except:
        print("索引页面获取异常")

In [3]:
def get_total_num(html):
    soup=BeautifulSoup(html,'html.parser')
    div=soup.find('div',attrs={'class':'search-result'})
    return div.span.string

In [4]:
date=time.strftime('%Y%m%d')
total_num=get_total_num(get_html('http://sh.lianjia.com/ershoufang/d1'))

In [5]:
def parse_index_page(date,total_num,index_page):
    path=os.path.join(os.getcwd(),'indexInfo.csv')
    fieldnames=['date','total_num','key','title','total_price','total_price_unit','per_price','per_price_unit','prop1']
    if  not os.path.exists(path):     #如果文件不存在，创建文件并写入表头
        csv_file=open(path,'w',newline='')
        writer=csv.DictWriter(csv_file,fieldnames=fieldnames)
        writer.writeheader()
        csv_file.close()
        
    csv_file=open(path,'a+',newline='')
    writer=csv.DictWriter(csv_file,fieldnames=fieldnames)
    
    
    item={}
    item.update({'date':date,'total_num':total_num})
    
    soup=BeautifulSoup(index_page,'html.parser')
    index_info=soup.find_all('a',attrs={'class':'text link-hover-green js_triggerGray js_fanglist_title'})
    for a in index_info:
        key=a['key']
        title=a['title']
        item.update({'key':key,'title':title})
        
        price_div=a.parent.next_sibling.next_sibling
        total_price=price_div.find('span',attrs={'class':'total-price strong-num'}).string
        total_price_unit=price_div.find('span',attrs={'class':'unit'}).string
        per_price_info=price_div.find('span',attrs={'class':'info-col price-item minor'}).string
        per_price_info=re.search('(\d+)(\S{3})',per_price_info).groups()
        per_price=per_price_info[0]
        per_price_unit=per_price_info[1]
        
        prop_div=price_div.next_sibling.next_sibling
        props=prop_div.find_all('span',attrs={'class':'c-prop-tag2'})
        prop1=props[0].string
#         prop2=props[1].string
#         prop3=props[2].string
        
        item.update({'total_price':total_price,'total_price_unit':total_price_unit,'per_price':per_price,
                    'per_price_unit':per_price_unit,'prop1':prop1})
        
        writer.writerow(item)    #定义writer2csv函数，每次调用函数写入item,结果是重复写入最后一个item，改用writer写入，无问题
        
        
    csv_file.close()

In [6]:
def parse_detail_page(detail_page):
    pass

In [7]:
def get_all_items():
    for i in range(1,30):
        url=url_base+str(i)
        index_page=get_html(url)
        parse_index_page(date,total_num,index_page)
        time.sleep(3)

In [8]:
def main():
    get_all_items()

main()